In [1]:
# coding=UTF-8   
import torch  
import torch.nn as nn  
import yaml
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
from custom_data_io import myset
import pandas as pd
import os

# def config(config_path):
#     """ Loading config file. """
#     with open(config_path, 'r') as f:
#         return yaml.load(f)
# hr = eval('models.cls_hrnet.get_cls_net')(config('experiments/cls_hrnet_w18_sgd_lr5e-2_wd1e-4_bs32_x100.yaml'))
# hr.load_state_dict(torch.load("hrnetv2_w18_imagenet_pretrained.pth"))
# hr.classifier = nn.Linear(2048, 3)

In [2]:
Train = '/home/mmlab206/datasets/芒果/train'
Test = '/home/mmlab206/Desktop/test_noperson'
Val = '/home/mmlab206/datasets/芒果/val'
val = '/home/mmlab206/datasets/芒果/val_noperson'
from RandAugment import RandAugment
# number of subprocesses to use for data loading
num_workers = 4
# how many samples per batch to load
batch_size = 16

class2idx = {'A': 0, 'B': 1, 'C': 2}

# convert data to a normalized torch.FloatTensor
train_transforms = transforms.Compose([transforms.Resize((385, 505)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize((385, 505)),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
train_transforms.transforms.insert(0, RandAugment(2, 14))
# choose the training and test datasets
train_data = datasets.ImageFolder(root = Train, transform=train_transforms)
valid_data = myset(Val, '/home/mmlab206/datasets/芒果/dev.csv', class2idx, test_transforms)
test_data = myset(Test, './test_example.csv', class2idx, test_transforms)

val_data = myset(val, '/home/mmlab206/datasets/芒果/dev.csv', class2idx, test_transforms)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size,  num_workers=num_workers,shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,  num_workers=num_workers,shuffle=False)

val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size,  num_workers=num_workers,shuffle=False)

In [15]:
result1 = []
for data, target in train_loader:
    result1.append(target)
result1 = torch.cat(result1, 0)
result2 = []
for data, target in valid_loader:
    result2.append(target)
result2 = torch.cat(result2, 0)
result3 = []
for data, target in test_loader:
    result3.append(target)
result3 = torch.cat(result3, 0)

In [51]:
class ensemble(nn.Module):
    def __init__(self):
        super(ensemble, self).__init__()   
        wA= nn.Linear(5, 1) 
        wB= nn.Linear(5, 1)
        wC= nn.Linear(5, 1)
        torch.nn.init.constant_(wA.weight, 1.0)
        torch.nn.init.constant_(wB.weight, 1.0)
        torch.nn.init.constant_(wC.weight, 1.0)
        self.wA = wA
        self.wB = wB
        self.wC = wC
    def forward(self, hr, alex, vgg, res, mob):
        wA = self.wA
        wB = self.wB
        wC = self.wC
        #print(wA.weight)
        A = torch.cat((hr[:,0].view(-1, 1), alex[:,0].view(-1, 1), vgg[:,0].view(-1, 1), res[:,0].view(-1, 1), mob[:,0].view(-1, 1)), 1)
        B = torch.cat((hr[:,1].view(-1, 1), alex[:,1].view(-1, 1), vgg[:,1].view(-1, 1), res[:,1].view(-1, 1), mob[:,1].view(-1, 1)), 1)
        C = torch.cat((hr[:,2].view(-1, 1), alex[:,2].view(-1, 1), vgg[:,2].view(-1, 1), res[:,2].view(-1, 1), mob[:,2].view(-1, 1)), 1)
        return torch.cat((wA(A), wB(B), wC(C)), 1) 

In [3]:
from FocalLoss import focal_loss
from tqdm.notebook import tqdm
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

class DML(object):
    def __init__(self, models, optimizer, parallel = False):
        self.models = models
        self.optimizer = optimizer
        self.loss_kl = nn.KLDivLoss(reduction='batchmean').cuda()  
        self.loss_fc = focal_loss(gamma = 3, alpha = [800/243, 800/293, 800/264]).cuda() 
        self.model_num = len(models)
        if parallel:
            for i in range(self.model_num):
                self.models[i] = nn.DataParallel(self.models[i])
    
    def train(self, n_epochs, train_loader, valid_loader, val_loss_min = np.Inf, resume = False):       
        if resume:
            for i in range(self.model_num):
                checkpoint = torch.load('DML' + str(i) + '.pth', map_location='cpu')
                self.models[i].load_state_dict(checkpoint, False)
        for i in range(self.model_num):
            self.models[i] = self.models[i].cuda()
            
        for epoch in range(1, n_epochs+1):
            print('running epoch: {}'.format(epoch))
            train_loss = self.train_one_epoch(epoch, train_loader)
            val_loss = self.validate(epoch, valid_loader)
            print('\tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                train_loss, val_loss))
            if val_loss <= val_loss_min:
                print('validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                val_loss_min,
                val_loss))
                for i in range(self.model_num):
                    torch.save(self.models[i].state_dict(), 'DML' + str(i) + '.pth')
                val_loss_min = val_loss
            
        writer.add_scalar("mango/train_loss", train_loss, epoch)
        writer.add_scalar("mango/val_loss", val_loss, epoch)
    
    def train_one_epoch(self, epoch, train_loader):
        train_loss = 0.0
        for i in range(self.model_num):
            self.models[i].train()
        with torch.autograd.set_detect_anomaly(True):
            for data, target in tqdm(train_loader):
                data, target = data.cuda(), target.cuda()
                data, target = Variable(data), Variable(target)
                outputs=[]
                for model in self.models:
                    outputs.append(model(data))
                self.optimizer.zero_grad()
                for i in range(self.model_num):
                    fc_loss = self.loss_fc(outputs[i], target)
                    kl_loss = 0
                    for j in range(self.model_num):
                        if i!=j:
                            kl_loss = kl_loss + self.loss_kl(F.log_softmax(outputs[i], dim = 1), 
                                                    F.softmax(Variable(outputs[j]), dim=1))
                    loss = fc_loss + kl_loss / (self.model_num - 1)
                    
                    loss.backward()
                    train_loss += loss.item()*data.size(0)
                    
                self.optimizer.step()
            train_loss = train_loss/len(train_loader.dataset)    
                
        return train_loss

    def validate(self, epoch, valid_loader):
        valid_loss = 0.0
        for data, target in tqdm(valid_loader):
            data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)            
            outputs=[]
            for model in self.models:
                outputs.append(model(data))
            for i in range(self.model_num):
                fc_loss = self.loss_fc(outputs[i], target)
                kl_loss = 0
                for j in range(self.model_num):
                    if i!=j:
                        kl_loss = kl_loss + self.loss_kl(F.log_softmax(outputs[i], dim = 1), 
                                                F.softmax(Variable(outputs[j]), dim=1))
                loss = fc_loss + kl_loss / (self.model_num - 1)
                
                valid_loss += loss.item()*data.size(0)
        valid_loss = valid_loss/len(valid_loader.dataset)
        return valid_loss

    def test(self, loader):
        valid_loss = 0.0
        for i in range(self.model_num):
            checkpoint = torch.load('DML' + str(i) + '.pth', map_location='cpu')
            self.models[i].load_state_dict(checkpoint, False)
            self.models[i].cuda()
            self.models[i].eval()
        correct = 0  
        total = 0
        for data, target in loader:
            data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)              
            outputs=[]
            for model in self.models:
                outputs.append(model(data))
            for i in range(self.model_num):
                fc_loss = self.loss_fc(outputs[i], target)
                kl_loss = 0
                for j in range(self.model_num):
                    if i!=j:
                        kl_loss = kl_loss + self.loss_kl(F.log_softmax(outputs[i], dim = 1), 
                                                F.softmax(Variable(outputs[j]), dim=1))
                loss = fc_loss + kl_loss / (self.model_num - 1)
                valid_loss += loss.item()*data.size(0)
            pred =  outputs[0] + outputs[1]
            pred = pred.data.max(1, keepdim=True)[1]
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
        valid_loss = valid_loss/len(valid_loader.dataset)
        print('Test Loss: {:.6f}'.format(valid_loss))

        print('Test Accuracy: %2d%% (%2d/%2d)' % (
            100. * correct / total, correct, total))

In [4]:
import torch.optim as optim
ms = []
model1 = models.resnet18(pretrained=True)
model1.fc = nn.Linear(512, 3)
ms.append(model1)
model2 = models.mobilenet_v2(pretrained=True)
model2.classifier[1] = nn.Linear(1280, 3)
ms.append(model2)
model3 = models.alexnet(pretrained=True)
model3.classifier._modules['6'] = nn.Linear(4096, 3)
ms.append(model3)

optimizer = optim.Adam([{'params': model1.parameters()}, {'params': model2.parameters()}, {'params': model3.parameters()}], lr = 0.00001, weight_decay=1e-3)

dml = DML(ms, optimizer, parallel = True)

Focal_loss alpha = [3.292181069958848, 2.7303754266211606, 3.0303030303030303], 将对每一类权重进行精细化赋值


In [ ]:
dml.train(300, train_loader, valid_loader)

running epoch: 1



	Training Loss: 2.633832 	Validation Loss: 1.672165
validation loss decreased (inf --> 1.672165).  Saving model ...
running epoch: 2



	Training Loss: 1.956547 	Validation Loss: 1.552349
validation loss decreased (1.672165 --> 1.552349).  Saving model ...
running epoch: 3



	Training Loss: 1.840605 	Validation Loss: 1.464986
validation loss decreased (1.552349 --> 1.464986).  Saving model ...
running epoch: 4



	Training Loss: 1.760438 	Validation Loss: 1.402730
validation loss decreased (1.464986 --> 1.402730).  Saving model ...
running epoch: 5


...


	Training Loss: 0.829416 	Validation Loss: 0.982632
running epoch: 116



	Training Loss: 0.840758 	Validation Loss: 0.928855
running epoch: 117



	Training Loss: 0.832951 	Validation Loss: 1.036125
running epoch: 118



	Training Loss: 0.832404 	Validation Loss: 0.933137
running epoch: 119



	Training Loss: 0.822965 	Validation Loss: 0.907008
running epoch: 120


In [8]:
dml.test(valid_loader)

Test Loss: 0.914517
Test Accuracy: 83% (667/800)


In [23]:
import pandas as pd
alex1 = pd.read_csv('csv/train/alex.csv')
res1 = pd.read_csv('csv/train/resnet.csv')
vgg1 = pd.read_csv('csv/train/vgg.csv')
hr1 = pd.read_csv('csv/train/hr.csv')
mob1 = pd.read_csv('csv/train/mob.csv')
alex2 = pd.read_csv('csv/val/alex.csv')
res2 = pd.read_csv('csv/val/resnet.csv')
vgg2 = pd.read_csv('csv/val/vgg.csv')
hr2 = pd.read_csv('csv/val/hr.csv')
mob2= pd.read_csv('csv/val/mob.csv')
#squeeze = pd.read_csv('squeeze.csv')
from tqdm.notebook import tqdm
loss_min = 0.4
batch = 80
ensem = ensemble()
optimizer = torch.optim.Adam([{'params': ensem.parameters()}],lr = 1e-4 , weight_decay=1e-2)
criterion = FocalLoss().cuda()
ensem.cuda()
n_epochs = 1000

for epoch in range(1, n_epochs+1):
    train_loss = 0.0
    val_loss = 0.0
    print('running epoch: {}'.format(epoch))
    ensem.train()
    for i in range(0, len(result1), batch):
        optimizer.zero_grad()
        target = result1[i:i+80]
        target = target.cuda()
                # Forward prop.
        output1 = torch.tensor(hr1.values[i:i+80,]).cuda()
        output2 = torch.tensor(alex1.values[i:i+80,]).cuda()
        output3 = torch.tensor(vgg1.values[i:i+80,]).cuda()
        output4 = torch.tensor(res1.values[i:i+80,]).cuda() 
        output5 = torch.tensor(mob1.values[i:i+80,]).cuda() 
        output = ensem(output1.float(), output2.float(), output3.float(), output4.float(), output5.float())
        # Calculate loss
        loss = criterion(output, target)

        loss.backward()
        
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('train_loss: ', train_loss)
    ensem.eval()
    for i in range(0, len(result2), batch):
        optimizer.zero_grad()
        target = result2[i:i+80]
        target = target.cuda()
                # Forward prop.
        output1 = torch.tensor(hr2.values[i:i+80,]).cuda()
        output2 = torch.tensor(alex2.values[i:i+80,]).cuda()
        output3 = torch.tensor(vgg2.values[i:i+80,]).cuda()
        output4 = torch.tensor(res2.values[i:i+80,]).cuda() 
        output5 = torch.tensor(mob2.values[i:i+80,]).cuda() 
        output = ensem(output1.float(), output2.float(), output3.float(), output4.float(), output5.float())
        # Calculate loss
        loss2 = criterion(output, target)
        val_loss += loss2.item()*data.size(0)
    val_loss = val_loss/len(valid_loader.dataset)
    print('val_loss: ', val_loss)
    if val_loss <= loss_min:
        loss_min = val_loss
        torch.save(ensem, 'ensem.pth')
        print('save')

running epoch: 1


/home/mmlab206/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


train_loss:  0.6399870793077741
val_loss:  0.8275227844715118
running epoch: 2
train_loss:  0.6359500726938301
val_loss:  0.8220354437828064
running epoch: 3
train_loss:  0.6322457080462106
val_loss:  0.8166169106960297
running epoch: 4
train_loss:  0.6285778681484534
val_loss:  0.8112451374530792
running epoch: 5
train_loss:  0.624940221669366
val_loss:  0.8059158951044083
running epoch: 6
train_loss:  0.621331217531811
val_loss:  0.8006271332502365
running epoch: 7
train_loss:  0.6177498354296397
val_loss:  0.7953776061534882
running epoch: 8
train_loss:  0.6141951975966535
val_loss:  0.7901664674282074
running epoch: 9
train_loss:  0.6106666537859408
val_loss:  0.7849928438663483
running epoch: 10
train_loss:  0.6071634355770505
val_loss:  0.779856052994728
running epoch: 11
train_loss:  0.60368501677628
val_loss:  0.7747552573680878
running epoch: 12
train_loss:  0.6002306150726665
val_loss:  0.7696897894144058
running epoch: 13
train_loss:  0.5967997779589959
val_loss:  0.76465911

train_loss:  0.37575120556727337
val_loss:  0.43987224251031876
running epoch: 105
train_loss:  0.37448973943115405
val_loss:  0.43795647770166396
running epoch: 106
train_loss:  0.3732561642489238
val_loss:  0.43608138859272005
running epoch: 107
train_loss:  0.3720503452736217
val_loss:  0.4342468813061714
running epoch: 108
train_loss:  0.37087213391039836
val_loss:  0.43245290219783783
running epoch: 109
train_loss:  0.3697212727514503
val_loss:  0.43069922775030134
running epoch: 110
train_loss:  0.3685976856428644
val_loss:  0.42898581326007845
running epoch: 111
train_loss:  0.36750103091499503
val_loss:  0.4273124158382416
running epoch: 112
train_loss:  0.3664311361796311
val_loss:  0.4256788149476051
running epoch: 113
train_loss:  0.36538773379933887
val_loss:  0.42408488392829896
running epoch: 114
train_loss:  0.3643705634868633
val_loss:  0.4225303366780281
running epoch: 115
train_loss:  0.3633792543802254
val_loss:  0.4210148945450783
running epoch: 116
train_loss:  0.3

/home/mmlab206/anaconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type ensemble. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


...

train_loss:  0.3156774179636295
val_loss:  0.40140083283185957
running epoch: 535
train_loss:  0.31566530619524164
val_loss:  0.4014341324567795
running epoch: 536
train_loss:  0.3156532635336882
val_loss:  0.4014673486351967
running epoch: 537
train_loss:  0.3156412982573923
val_loss:  0.40150035470724105
running epoch: 538
train_loss:  0.31562939814762886
val_loss:  0.4015332579612732
running epoch: 539
train_loss:  0.31561757587777306
val_loss:  0.40156601518392565
running epoch: 540
train_loss:  0.3156058040362049
val_loss:  0.4015986815094948
running epoch: 541
train_loss:  0.31559409664130655
val_loss:  0.40163118690252303
running epoch: 542
train_loss:  0.31558248047955373
val_loss:  0.4016635537147522
running epoch: 543
train_loss:  0.31557094035805133
val_loss:  0.4016957595944405
running epoch: 544
train_loss:  0.3155594473875663
val_loss:  0.4017278358340263
running epoch: 545
train_loss:  0.31554804803714054
val_loss:  0.40175979137420653
running epoch: 546
train_loss:  0.3

train_loss:  0.314755118964678
val_loss:  0.40414829552173615
running epoch: 636
train_loss:  0.3147484431270119
val_loss:  0.4041700974106789
running epoch: 637
train_loss:  0.31474179166996413
val_loss:  0.4041917994618416
running epoch: 638
train_loss:  0.3147351944110123
val_loss:  0.4042134404182434
running epoch: 639
train_loss:  0.3147286173271642
val_loss:  0.4042349919676781
running epoch: 640
train_loss:  0.31472207045861245
val_loss:  0.40425641685724256
running epoch: 641
train_loss:  0.3147155608145481
val_loss:  0.40427776575088503
running epoch: 642
train_loss:  0.31470912355458863
val_loss:  0.4042989909648895
running epoch: 643
train_loss:  0.3147027041207068
val_loss:  0.404320153594017
running epoch: 644
train_loss:  0.31469630959786865
val_loss:  0.4043412506580353
running epoch: 645
train_loss:  0.31468996620423734
val_loss:  0.40436225533485415
running epoch: 646
train_loss:  0.31468365768606715
val_loss:  0.4043831929564476
running epoch: 647
train_loss:  0.31467

train_loss:  0.31423838147521777
val_loss:  0.40595313161611557
running epoch: 737
train_loss:  0.31423456120090937
val_loss:  0.4059675708413124
running epoch: 738
train_loss:  0.31423076920963383
val_loss:  0.4059819206595421
running epoch: 739
train_loss:  0.31422700481941584
val_loss:  0.40599623769521714
running epoch: 740
train_loss:  0.31422324332759305
val_loss:  0.40601049810647966
running epoch: 741
train_loss:  0.31421950958837785
val_loss:  0.4060246840119362
running epoch: 742
train_loss:  0.3142157986763928
val_loss:  0.40603883415460584
running epoch: 743
train_loss:  0.31421210559048546
val_loss:  0.4060529306530952
running epoch: 744
train_loss:  0.31420844406488135
val_loss:  0.4060669347643852
running epoch: 745
train_loss:  0.3142047893779744
val_loss:  0.4060808762907982
running epoch: 746
train_loss:  0.3142011825619476
val_loss:  0.40609477609395983
running epoch: 747
train_loss:  0.3141975596247815
val_loss:  0.4061086043715477
running epoch: 748
train_loss:  0.

train_loss:  0.3139401138957875
val_loss:  0.40715658813714983
running epoch: 838
train_loss:  0.31393790226870144
val_loss:  0.40716626644134524
running epoch: 839
train_loss:  0.3139356869286385
val_loss:  0.4071759030222893
running epoch: 840
train_loss:  0.31393348598583276
val_loss:  0.4071854993700981
running epoch: 841
train_loss:  0.31393129582202617
val_loss:  0.4071950912475586
running epoch: 842
train_loss:  0.3139291147322804
val_loss:  0.4072046563029289
running epoch: 843
train_loss:  0.3139269412958134
val_loss:  0.40721418857574465
running epoch: 844
train_loss:  0.31392476708265227
val_loss:  0.4072236746549606
running epoch: 845
train_loss:  0.31392262067893006
val_loss:  0.40723315328359605
running epoch: 846
train_loss:  0.3139204712442063
val_loss:  0.40724258273839953
running epoch: 847
train_loss:  0.31391834134049856
val_loss:  0.4072520017623901
running epoch: 848
train_loss:  0.3139162235986844
val_loss:  0.4072613701224327
running epoch: 849
train_loss:  0.31

train_loss:  0.31376371667368735
val_loss:  0.40797597318887713
running epoch: 939
train_loss:  0.3137623874658608
val_loss:  0.407982636988163
running epoch: 940
train_loss:  0.3137610704199279
val_loss:  0.4079892888665199
running epoch: 941
train_loss:  0.3137597585077538
val_loss:  0.4079959124326706
running epoch: 942
train_loss:  0.31375846540673286
val_loss:  0.40800253450870516
running epoch: 943
train_loss:  0.31375716292855094
val_loss:  0.4080091044306755
running epoch: 944
train_loss:  0.31375586096185054
val_loss:  0.40801565200090406
running epoch: 945
train_loss:  0.313754587751777
val_loss:  0.4080221578478813
running epoch: 946
train_loss:  0.31375331662551703
val_loss:  0.40802867114543917
running epoch: 947
train_loss:  0.31375204358593733
val_loss:  0.40803513675928116
running epoch: 948
train_loss:  0.31375078113591914
val_loss:  0.40804161727428434
running epoch: 949
train_loss:  0.313749517738713
val_loss:  0.40804801136255264
running epoch: 950
train_loss:  0.31

In [1]:
import pandas as pd
import numpy as np
alex = pd.read_csv('/home/mmlab206/Desktop/alex.csv')
res = pd.read_csv('/home/mmlab206/Desktop/res.csv')
hr = pd.read_csv('/home/mmlab206/Desktop/hr.csv')
mob = pd.read_csv('/home/mmlab206/Desktop/mob.csv')

output =  mob.values

answer = []
for i in range(1600):
    #answer.append(np.argmax(output[i,:].cpu().data.numpy()))
    answer.append(np.argmax(output[i,:]))

In [2]:
class2idx = {'A': 0, 'B': 1, 'C': 2}
idx2class = {class2idx[k]: k for k in class2idx.keys()}
a = []

for i in range(len(answer)):
    a.append(idx2class.get(answer[i]))

In [3]:
ans = pd.read_csv('./test_example.csv')
ans['label'] = a
ans.to_csv('/home/mmlab206/Desktop/test_example.csv',index=0)
ans

,image_id,label
0,00001.jpg,C
1,00006.jpg,A
2,00011.jpg,B
3,00016.jpg,A
4,00025.jpg,C
...,...,...
1595,07977.jpg,B
1596,07991.jpg,C
1597,07992.jpg,A
1598,07994.jpg,B


In [52]:
import pandas as pd
from FocalLoss import focal_loss
alex = pd.read_csv('csv/test/alex.csv')
res = pd.read_csv('csv/test/resnet.csv')
#vgg = pd.read_csv('csv/test/vgg.csv')
hr = pd.read_csv('csv/test/hr.csv')
mob = pd.read_csv('csv/test/mob.csv')
ensem = ensemble()
ensem = torch.load('ensem.pth')
#criterion = focal_loss(gamma = 3, alpha = [800/243, 800/293, 800/264]).cuda()

# result3 = []
# for data, target in test_loader:
#     result3.append(target)
# result3 = torch.cat(result3, 0)

output1 = torch.tensor(hr.values).cuda()
output2 = torch.tensor(alex.values).cuda()
output3 = torch.tensor((alex.values/6 + res.values + hr.values + mob.values)/4).cuda()
output4 = torch.tensor(res.values).cuda() 
output5 = torch.tensor(mob.values).cuda() 
output = ensem(output1.float(), output2.float(), output3.float(), output4.float(), output5.float())
# Calculate loss
#loss = criterion(output, result3.cuda())
#print(loss)
#output = alex.values + res.values + vgg.values + hr.values + mob.values


answer = []
for i in range(800):
    #answer.append(np.argmax(output[i,:].cpu().data.numpy()))
    answer.append(np.argmax(output[i,:]))

targ = result3

k = 0
for i in range(800):
    if(answer[i] != int(targ[i])):
        k += 1
c = 800 - k      
from sklearn.metrics import accuracy_score
print(f"Test Accuracy: {accuracy_score(targ, answer)} ({c}/800)")

from sklearn.metrics import recall_score
print('WAR ', recall_score(targ, answer, average = 'weighted'))

Focal_loss alpha = [3.292181069958848, 2.7303754266211606, 3.0303030303030303], 将对每一类权重进行精细化赋值


NameError: name 'result3' is not defined

In [99]:
A = 0
for i in range(243):
    if answer[i] == 0:
        A += 1
A = A/243
B = 0
for i in range(243, 536, 1):
    if answer[i] == 1:
        B += 1
B = B/293
C = 0
for i in range(536, 800, 1):
    if answer[i] == 2:
        C += 1
C = C/264

In [100]:
A/3+B/3+C/3

0.8432175481695138

In [101]:
A*(243/800)+B*(293/800)+C*(264/800)

0.84